In [1]:
import numpy as np
import nilearn
import pandas as pd
import os
import pycaret
from nilearn.connectome import sym_matrix_to_vec
import pickle

import json
from modeling_utils import *
from pycaret.classification import *

# Prepare Data for Pycaret

In [2]:
pca_features = np.load("group_pca_features.npy")

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
z_pca_features = scaler.fit_transform(pca_features)

In [3]:
print(pca_features.shape)

(936, 936)


In [7]:
data_paths = json.load(open('data_paths.json', 'r'))

In [8]:
participants = pd.read_csv("participants.csv")

In [9]:
pca_features = pd.DataFrame(pca_features)


In [10]:
z_pca_features = pd.DataFrame(z_pca_features)

In [11]:
pca_features["diagnosis"] = participants["diagnosis"].values

In [13]:
z_pca_features["diagnosis"] = participants["diagnosis"].values

In [14]:
z_pca_features

,0,1,2,3,4,5,6,7,8,9,...,927,928,929,930,931,932,933,934,935,diagnosis
0,-0.037451,0.877742,0.851664,1.112781,-0.018495,1.495365,-0.928388,0.402388,0.998005,-3.025522,...,1.795801,1.077133,0.709689,-0.783696,0.394100,1.399889,-0.640550,-0.935044,-0.540477,1
1,-0.788723,-0.451073,1.064654,-1.862337,1.165596,-0.573462,0.844521,-0.820587,1.161539,-2.357456,...,0.619354,-0.213989,-1.205781,0.544656,-1.019299,-0.702494,0.892512,0.821963,-1.836511,1
2,-0.881122,-0.537879,-0.295877,0.944804,-0.444080,-1.085741,0.022726,0.769365,0.046054,-1.314172,...,0.992217,0.232463,-0.471629,-0.604260,-1.051697,0.155915,-1.153250,0.716743,-0.020210,1
3,-0.037001,-0.744177,-0.141481,-0.251439,0.515631,-1.574424,-0.851996,-0.685689,0.158716,-0.547914,...,-0.885186,2.270919,-0.253184,-0.082594,0.340459,1.015177,-1.626901,-0.556460,-0.487620,1
4,0.091975,-1.233866,-0.599923,1.511242,-2.006953,-1.232552,0.415964,-0.587609,0.710219,-0.831671,...,-1.191025,-1.778869,-1.238162,1.419053,1.477573,0.484267,-0.977878,0.321335,-2.106146,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,0.363801,-1.202154,-0.094210,-1.330508,0.219739,0.003685,1.497834,-0.500771,1.097432,-0.475188,...,-1.459166,-0.828485,-1.878068,-1.019063,-1.968675,-1.196955,-0.856778,0.489474,-1.582312,1
932,0.408510,-0.459289,-1.362092,0.554097,0.312255,0.102078,2.286778,-1.079378,-1.124109,-0.955629,...,0.815948,-0.619265,0.553024,-0.443896,-0.939387,-0.619790,0.370538,-0.588213,0.146271,1
933,0.335936,-0.455098,-1.772064,0.311882,-1.042444,0.307232,1.922203,0.010731,-0.138276,-1.483103,...,-0.555294,2.339454,1.459950,1.965130,0.802706,1.123248,2.534214,-1.481015,1.589744,1
934,-0.235598,-0.984890,-0.643051,0.084609,-0.373247,-0.092442,-0.251151,-0.031585,-0.479438,-1.116687,...,-1.393623,-0.063707,-0.323181,-0.040437,-0.267129,0.798329,0.434839,-0.557839,-0.863458,1


# Model Selection

In [21]:
exp = setup(z_pca_features, target = 'diagnosis', session_id = 123)

,Description,Value
0,Session id,123
1,Target,diagnosis
2,Target type,Binary
3,Original data shape,"(936, 937)"
4,Transformed data shape,"(936, 937)"
5,Transformed train set shape,"(655, 937)"
6,Transformed test set shape,"(281, 937)"
7,Numeric features,936
8,Preprocess,True
9,Imputation type,simple


In [18]:
performance = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:52:47
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7709,0.7456,0.4418,0.5917,0.5044,0.3600,0.3672,0.1710
svm,SVM - Linear Kernel,0.7559,0.0000,0.2245,0.6058,0.3189,0.2123,0.2520,0.1460
gbc,Gradient Boosting Classifier,0.7496,0.7033,0.1951,0.5650,0.2857,0.1816,0.2188,1.6400
et,Extra Trees Classifier,0.7344,0.6667,0.0000,0.0000,0.0000,0.0000,0.0000,0.2860
knn,K Neighbors Classifier,0.7343,0.5813,0.0340,0.3500,0.0616,0.0298,0.0557,0.2910
rf,Random Forest Classifier,0.7329,0.6733,0.0000,0.0000,0.0000,-0.0030,-0.0076,0.3790
ada,Ada Boost Classifier,0.7084,0.6594,0.3346,0.4594,0.3803,0.1960,0.2046,0.5180
nb,Naive Bayes,0.6872,0.6733,0.3908,0.4122,0.3956,0.1868,0.1896,0.1730
ridge,Ridge Classifier,0.6489,0.0000,0.5333,0.3816,0.4441,0.1977,0.2043,0.1510
dt,Decision Tree Classifier,0.6398,0.5544,0.3729,0.3420,0.3545,0.1067,0.1074,0.2750


KeyboardInterrupt: 

In [12]:
performance =pull()
performance.to_csv("results/pca_model_results.csv")


# Feature Selection

In [23]:
lr = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7424,0.6843,0.4118,0.5000,0.4516,0.2854,0.2876
1,0.7879,0.6806,0.4444,0.6667,0.5333,0.4031,0.4170
2,0.8333,0.8310,0.6111,0.7333,0.6667,0.5568,0.5609
3,0.7424,0.7569,0.3333,0.5455,0.4138,0.2609,0.2739
4,0.8182,0.7789,0.5000,0.7500,0.6000,0.4884,0.5052
5,0.8154,0.8480,0.5294,0.6923,0.6000,0.4828,0.4901
6,0.7846,0.8100,0.5294,0.6000,0.5625,0.4204,0.4218
7,0.7692,0.7341,0.4118,0.5833,0.4828,0.3399,0.3484
8,0.7231,0.6593,0.3529,0.4615,0.4000,0.2241,0.2275


In [24]:
feature_importance = pd.DataFrame({'Feature': np.array(get_config('X_test').columns).astype(int), 'Importance': abs(lr.coef_[0])})


In [ ]:
feature_importance

In [25]:
feature_importance["Importance"].min()
    

9.221089726456163e-05

In [27]:
 feature_importance["Importance"].max()


0.7711673765109432

In [158]:
np.save('results/component_importance.npy', feature_importance["Importance"].values)

SyntaxError: EOL while scanning string literal (3861651521.py, line 1)

In [28]:
eigenvectors = abs(np.load("group_pca_eigenvectors.npy"))

In [29]:
feat_importance_mat = feature_importance["Importance"].values * eigenvectors.T

In [31]:
feat_importance_mat.shape

(1700500, 936)

In [32]:
np.save('results/z_feature_importance_matrix.npy', feat_importance_mat)

In [61]:
all_features = np.load('all_features.npy')
feature_labels = np.load('feature_labels.npy')

In [62]:
scaler = StandardScaler()
z_all_features = scaler.fit_transform(all_features)
np.save('z_all_features.npy', z_all_features)

In [64]:
z_all_features = pd.DataFrame(z_all_features)

In [37]:
feat_importance_mat = np.load('results/feature_importance_matrix.npy')
z_feat_importance_mat = np.load('results/z_feature_importance_matrix.npy')

In [44]:
best_features = get_n_best_features(feat_importance_mat, 1000, all_features, feature_labels)

In [43]:
feat_importance_mat

array([[1.32625084e-05, 3.12879371e-05, 4.66042814e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.15473699e-05, 3.24204194e-05, 3.72083719e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.50533326e-05, 1.71357340e-05, 9.86628015e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.10818214e-07, 8.54103142e-06, 1.89452656e-06],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.92798257e-06, 1.21348639e-05, 3.13698906e-06],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.12861588e-06, 2.94646982e-05, 8.79953359e-06]])

In [42]:
z_feat_importance_mat

array([[3.05214219e-04, 2.13681379e-04, 5.35857306e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.65743205e-04, 2.21415681e-04, 4.27822881e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.46427012e-04, 1.17028721e-04, 1.13442760e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        8.14210816e-06, 1.85266425e-05, 3.33923275e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.05048995e-05, 2.63221471e-05, 5.52915796e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.39130703e-04, 6.39128814e-05, 1.55097803e-04]])

In [65]:
z_best_features = get_n_best_features(z_feat_importance_mat, 1000, z_all_features, feature_labels)

In [52]:
best_features.values == z_best_features.values

array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]])

In [53]:
z_best_features.values

array([[-2.76871544, -3.09602424, -3.38060971, ...,  2.70305203,
         1.07287186,  0.84343466],
       [-0.83250696, -1.13576647, -1.29630891, ...,  2.77670981,
         0.55735037,  0.3127236 ],
       [ 0.14768757,  0.39807936,  0.62503312, ..., -0.55777919,
         0.71616216,  0.7690363 ],
       ...,
       [-2.01000404, -1.8392076 , -2.42936382, ...,  0.44546627,
         0.75621637,  1.27259654],
       [-0.95856868, -0.65262072, -1.53780944, ...,  0.77340449,
         0.75496435,  0.45636082],
       [-1.88806156, -1.10484479, -1.18756825, ...,  0.86996625,
         0.17837424,  0.936723  ]])

In [66]:
z_best_features

,grad_comb_200_aligned_128605,grad_comb_200_aligned_131005,grad_comb_200_aligned_132005,grad_comb_200_aligned_124404,grad_comb_200_aligned_24805,grad_comb_200_aligned_129605,grad_comb_200_aligned_130605,grad_comb_200_aligned_131400,grad_comb_200_aligned_29600,grad_comb_200_aligned_133000,grad_comb_200_aligned_126805,grad_comb_200_aligned_131405,grad_comb_200_aligned_27405,grad_comb_200_aligned_123604,grad_comb_200_aligned_23804,grad_comb_200_aligned_130205,grad_comb_200_aligned_28000,grad_comb_200_aligned_117802,grad_comb_200_aligned_131605,grad_comb_200_aligned_24804,grad_comb_200_aligned_119602,grad_comb_200_aligned_125604,grad_comb_200_aligned_129205,grad_comb_200_aligned_30600,grad_comb_200_aligned_23805,grad_comb_200_aligned_130005,grad_comb_200_aligned_23202,grad_comb_200_aligned_128405,grad_comb_200_aligned_121200,grad_comb_200_aligned_136200,grad_comb_200_aligned_33800,grad_comb_200_aligned_112801,grad_comb_200_aligned_22602,grad_comb_200_aligned_133005,grad_comb_200_aligned_17802,grad_comb_200_aligned_135000,grad_comb_200_aligned_26005,grad_comb_200_aligned_121202,grad_comb_200_aligned_23204,grad_comb_200_aligned_21202,grad_comb_200_aligned_28005,grad_comb_200_aligned_27404,grad_comb_200_aligned_122402,grad_comb_200_aligned_129200,grad_comb_200_aligned_26405,grad_comb_200_aligned_25604,grad_comb_200_aligned_132805,grad_comb_200_aligned_22804,grad_comb_200_aligned_119802,grad_comb_200_aligned_125605,grad_comb_200_aligned_128205,grad_comb_200_aligned_131004,grad_comb_200_aligned_120400,grad_comb_200_aligned_125802,grad_comb_200_aligned_124604,grad_comb_200_aligned_26404,grad_comb_200_aligned_24005,grad_comb_200_aligned_27605,grad_comb_200_aligned_124602,grad_comb_200_aligned_26004,grad_comb_200_aligned_21002,grad_comb_200_aligned_24004,grad_comb_200_aligned_132000,grad_comb_200_aligned_123602,grad_comb_200_aligned_131600,grad_comb_200_aligned_113801,grad_comb_200_aligned_32200,grad_comb_200_aligned_131404,grad_comb_200_aligned_131805,grad_comb_200_aligned_33600,grad_comb_200_aligned_106201,grad_comb_200_aligned_34200,grad_comb_200_aligned_125804,grad_comb_200_aligned_114001,grad_comb_200_aligned_25605,grad_comb_200_aligned_124406,grad_comb_200_aligned_29605,grad_comb_200_aligned_136600,grad_comb_200_aligned_19002,grad_comb_200_aligned_123204,grad_comb_200_aligned_27400,grad_comb_200_aligned_18402,grad_comb_200_aligned_120402,grad_comb_200_aligned_14801,grad_comb_200_aligned_29205,grad_comb_200_aligned_127205,grad_comb_200_aligned_127805,grad_comb_200_aligned_23412,grad_comb_200_aligned_136400,grad_comb_200_aligned_7401,grad_comb_200_aligned_16804,grad_comb_200_aligned_23206,grad_comb_200_aligned_22604,grad_comb_200_aligned_24002,grad_comb_200_aligned_29400,grad_comb_200_aligned_126804,grad_comb_200_aligned_130604,grad_comb_200_aligned_119804,grad_comb_200_aligned_128601,grad_comb_200_aligned_30400,grad_comb_200_aligned_23404,grad_comb_200_aligned_136000,grad_comb_200_aligned_29200,grad_comb_200_aligned_13601,grad_comb_200_aligned_28004,grad_comb_200_aligned_26800,grad_comb_200_aligned_23002,grad_comb_200_aligned_129005,grad_comb_200_aligned_127804,grad_comb_200_aligned_133600,grad_comb_200_aligned_32800,grad_comb_200_aligned_3601,grad_comb_200_aligned_30605,grad_comb_200_aligned_130600,grad_comb_200_aligned_23004,grad_comb_200_aligned_11001,grad_comb_200_aligned_23405,grad_comb_200_aligned_128600,grad_comb_200_aligned_22812,grad_comb_200_aligned_124004,grad_comb_200_aligned_22806,grad_comb_200_aligned_19402,grad_comb_200_aligned_33400,grad_comb_200_aligned_123002,grad_comb_200_aligned_124402,grad_comb_200_aligned_27401,grad_comb_200_aligned_22805,grad_comb_200_aligned_122002,grad_comb_200_aligned_27004,grad_comb_200_aligned_17804,grad_comb_200_aligned_119002,grad_comb_200_aligned_130001,grad_comb_200_aligned_115401,grad_comb_200_aligned_27000,grad_comb_200_aligned_28205,grad_comb_200_aligned_23205,grad_comb_200_aligned_24202,grad_comb_200_aligned_104401,grad_comb_200_aligned_23212,grad_comb_200_aligned_17602,grad_comb_2

In [67]:
z_best_features["diagnosis"] = participants['diagnosis'].values

In [10]:
best_features = pd.read_csv("best_features/1000_best_features.csv")
best_features["diagnosis"] = participants["diagnosis"]

In [71]:
exp2 = setup(z_best_features, target = 'diagnosis', session_id = 123)


,Description,Value
0,Session id,123
1,Target,diagnosis
2,Target type,Binary
3,Original data shape,"(936, 1001)"
4,Transformed data shape,"(936, 1001)"
5,Transformed train set shape,"(655, 1001)"
6,Transformed test set shape,"(281, 1001)"
7,Numeric features,1000
8,Preprocess,True
9,Imputation type,simple


In [72]:
best_model = compare_models(n_select = 1)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
svm,SVM - Linear Kernel,0.7710,0.0000,0.3523,0.6177,0.4380,0.3134,0.3355,0.2080
et,Extra Trees Classifier,0.7695,0.7588,0.1549,0.8233,0.2513,0.1942,0.2891,0.3080
lr,Logistic Regression,0.7665,0.7854,0.4833,0.5826,0.5213,0.3698,0.3769,1.4390
rf,Random Forest Classifier,0.7588,0.7406,0.1889,0.6994,0.2889,0.1976,0.2630,0.3730
lightgbm,Light Gradient Boosting Machine,0.7587,0.7614,0.2873,0.5980,0.3834,0.2577,0.2860,87.9480
gbc,Gradient Boosting Classifier,0.7495,0.7656,0.3088,0.5697,0.3905,0.2515,0.2740,1.7270
knn,K Neighbors Classifier,0.7466,0.6526,0.1670,0.5800,0.2525,0.1573,0.2018,0.3240
ada,Ada Boost Classifier,0.7344,0.7223,0.4193,0.5094,0.4561,0.2833,0.2877,0.5540
dummy,Dummy Classifier,0.7344,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2860
lda,Linear Discriminant Analysis,0.7223,0.7350,0.5480,0.4727,0.5026,0.3129,0.3182,0.2530


In [43]:
exp3 = setup(grad1_features, target = 'diagnosis', session_id = 123)

,Description,Value
0,Session id,123
1,Target,diagnosis
2,Target type,Binary
3,Original data shape,"(936, 1001)"
4,Transformed data shape,"(936, 1001)"
5,Transformed train set shape,"(655, 1001)"
6,Transformed test set shape,"(281, 1001)"
7,Numeric features,1000
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best_model = compare_models(n_select = 1)

In [16]:
performance = pull()
performance = performance[performance['Model'] != 'Dummy Classifier']
imp_features_mean_acc = performance['Accuracy'].mean()
imp_features_mean_auc = performance.loc[performance['AUC'] != 0, 'AUC'].mean()

In [17]:
performance[performance["Model"] != "Dummy Classifier"]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7341,0.7821,0.4432,0.8021,0.5503,0.3872,0.4331,0.870
rf,Random Forest Classifier,0.7083,0.7457,0.4258,0.6857,0.5156,0.3298,0.3513,0.459
gbc,Gradient Boosting Classifier,0.7082,0.7331,0.4765,0.6863,0.5548,0.3484,0.3667,0.454
xgboost,Extreme Gradient Boosting,0.6857,0.7298,0.4871,0.6361,0.5360,0.3094,0.3242,0.735
lightgbm,Light Gradient Boosting Machine,0.6758,0.7315,0.4932,0.6044,0.5272,0.2896,0.3025,0.657
knn,K Neighbors Classifier,0.6662,0.6867,0.4091,0.5730,0.4687,0.2440,0.2519,0.453
ada,Ada Boost Classifier,0.6595,0.6898,0.5697,0.5816,0.5651,0.2886,0.2956,0.443
nb,Naive Bayes,0.6406,0.7122,0.6530,0.5274,0.5771,0.2713,0.2834,0.659
lr,Logistic Regression,0.6339,0.6591,0.5121,0.5430,0.5191,0.2266,0.2309,0.484
svm,SVM - Linear Kernel,0.6276,0.0000,0.4879,0.5041,0.4920,0.2026,0.2017,0.418


In [18]:
imp_features_mean_auc

0.6701153846153847

In [19]:
p_mean_acc, p_mean_auc = random_feature_test(5, all_features, imp_features_mean_acc, imp_features_mean_auc, participants, n_tests = 5, workers = -1) # parallelize it

Fitting models on randomly picked features...
Test 1...
Test 2...
Test 3...
Test 4...
Test 5...


In [97]:
max_indices = np.argsort(-abs(eigenvectors[624]))[:500]

In [98]:
best_features = all_features.iloc[:, max_indices]
best_features["diagnosis"] = participants['diagnosis'].values


In [99]:
best_features

,1699658,1700181,1698658,1693658,1688658,1683658,1678658,1673658,1668658,1663658,1658658,1700306,1653658,1648658,1643658,1638658,1633658,1628658,1623658,1618658,1613658,1699181,1694181,1689181,1608658,1684181,1603658,1679181,1674181,1669181,1598658,1593658,1588658,1583658,1578658,1664181,1659181,1700123,1573658,1654181,1568658,1649181,1644181,1563658,1639181,1558658,1634181,1699306,1553658,1694306,1548658,1629181,1543658,1689306,1699616,1624181,1538658,1684306,1533658,1679306,1528658,1523658,1518658,1619181,1513658,1614181,1508658,1674306,1669306,1503658,1697658,1609181,1664306,1498658,1659306,1692658,1654306,1687658,1604181,1649306,1682658,1599181,1493658,1677658,1488658,1594181,1672658,1589181,1644306,1639306,1634306,1483658,1584181,1478658,1579181,1473658,1574181,1667658,1569181,1629306,1468658,1662658,1463658,1564181,1458658,1657658,1559181,1554181,1453658,1624306,1549181,1448658,1619306,1652658,1443658,1647658,1614306,1438658,1433658,1609306,1642658,1544181,1428658,1539181,1423658,1534181,1529181,1524181,1519181,1514181,1509181,1637658,1418658,1504181,1604306,1632658,1413658,1499181,1408658,1627658,1494181,1403658,1489181,1622658,1484181,1398658,1617658,1599306,1479181,1393658,1612658,1388658,1594306,1699123,1698306,1474181,1383658,1589306,1694123,1378658,1584306,1693306,1469181,1373658,1464181,1688306,1698181,1689123,1368658,1459181,1607658,1684123,1683306,1363658,1579306,1602658,1454181,1679123,1678306,1574306,1674123,1693181,1358658,1569306,1669123,1449181,1664123,1597658,1659123,1688181,1592658,1564306,1587658,1654123,1582658,1444181,1683181,1577658,1439181,1673306,1649123,1678181,1353658,1668306,1634123,1644123,1559306,1639123,1348658,1673181,1572658,1434181,1554306,1668181,1343658,1629123,1663306,1567658,1624123,1429181,1619123,1663181,1658306,1549306,1614123,1653306,1424181,1658181,1609123,1562658,1604123,1648306,1653181,1698616,1419181,1414181,1648181,1557658,1544306,1643306,1693616,1409181,1638306,1338658,1643181,1633306,1552658,1404181,1638181,1547658,...,1688616,1534306,1542658,1633181,1399181,1333658,1537658,1394181,1628306,1683616,1532658,1328658,1599123,1673616,1678616,1389181,1668616,1594123,1527658,1323658,1663616,1658616,1699733,1522658,1384181,1628181,1517658,1529306,1512658,1653616,1589123,1623181,1379181,1318658,1507658,1584123,1623306,1648616,1579123,1374181,1643616,1524306,1313658,1638616,1633616,1618306,1369181,1502658,1519306,1308658,1514306,1574123,1700150,1618181,1613306,1628616,1569123,1303658,1613181,1497658,1298658,1623616,1618616,1613616,1564123,1608306,1559123,1364181,1554123,1699600,1293658,1549123,1544123,1492658,1699706,1608181,1487658,1608616,1539123,1534123,1603616,1359181,1529123,1524123,1603181,1482658,1519123,1509306,1477658,1288658,1514123,1598181,1598616,1509123,1593616,1699537,1588616,1593181,1472658,1504123,1354181,1588181,1603306,1499123,1349181,1467658,1462658,1283658,1583181,1344181,1457658,1578181,1598306,1339181,1583616,1494123,1489123,1573181,1578616,1334181,1573616,1568181,1452658,1504306,1484123,1568616,1329181,1593306,1499306,1479123,1474123,1324181,1447658,1278658,1563181,1563616,1494306,1588306,1469123,1442658,1699541,1273658,1558181,1464123,1583306,1700128,1558616,1437658,1553616,1553181,1548616,1489306,1432658,1319181,1548181,1543616,1538616,1484306,1533616,1459123,1479306,1427658,1454123,1422658,1528616,1578306,1696658,1449123,1523616,1573306,1474306,1699634,1518616,1568306,1469306,1513616,1444123,1691658,1464306,1508616,1459306,1543181,1563306,1686658,1439123,1454306,1503616,1314181,1538181,1309181,1449306,1681658,1268658,1676658,1434123,1533181,1498616,1558306,1671658,1304181,1429123,1417658,1553306,1528181,1263658,1523181,1412658,1299181,1424123,1407658,1697306,1518181,1402658,1419123,1548306,1513181,1493616,1397658,1414123,1508181,1692306,1488616,1404123,1409123,1483616,1666658,1503181,1399123,1392658,1687306,1387658,1661658,1478616,1294181,1444306,1498181,1700182,1258658,1473616,1682306,1493181,1543306,1468616,1463616,1394123,1488181,1382658,1656658,1483181,dia

In [100]:
exp3 = setup(best_features, target = 'diagnosis', session_id = 123)


,Description,Value
0,Session id,123
1,Target,diagnosis
2,Target type,Binary
3,Original data shape,"(936, 501)"
4,Transformed data shape,"(936, 501)"
5,Transformed train set shape,"(655, 501)"
6,Transformed test set shape,"(281, 501)"
7,Numeric features,500
8,Preprocess,True
9,Imputation type,simple
